In [1]:
import os
import openai

In [2]:
from dotenv import load_dotenv
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from trulens_eval import Tru

tru = Tru()
tru.reset_database() # reset logged tables and logs

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


# Data

In [5]:
from llama_index import SimpleDirectoryReader

docs = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

from llama_index import Document

doc = Document(text="\n\n".\
                    join([doc.text for doc in docs]))

# Triad of Metrics

## Setup

In [6]:
from utils_triad_of_metrics import build_sentence_window_index

from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1) 

# Parser 
sentence_index = build_sentence_window_index(
    doc,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

In [7]:
from utils_triad_of_metrics import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

In [8]:
response = sentence_window_engine.query(
    "How do you create your AI portfolio?")
response.response

'To create an AI portfolio, you should showcase your expertise and experience in the field of artificial intelligence. This can include highlighting your knowledge of various AI technologies, programming languages, and algorithms. Additionally, you can include any AI projects you have worked on, such as developing machine learning models or implementing natural language processing systems. It is also beneficial to demonstrate your problem-solving skills and ability to apply AI techniques to real-world scenarios. Overall, your AI portfolio should provide a comprehensive overview of your AI skills and accomplishments.'

## Feedback Function

In [9]:
import nest_asyncio

nest_asyncio.apply() # for app

In [10]:
from trulens_eval import OpenAI as fOpenAI

provider = fOpenAI() # the model for the evaluator llm

### Answer Relevance

In [11]:
from trulens_eval import Feedback

f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons,
    name="Answer Relevance"
).on_input().on_output()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


### Context Relevance

In [12]:
from trulens_eval import TruLlama

context_selection = TruLlama.select_source_nodes().node.text

In [13]:
import numpy as np

f_qs_relevance = (
    Feedback(provider.qs_relevance,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input statement will be set to __record__.app.query.rets.source_nodes[:].node.text .


### Groundedness

In [14]:
from trulens_eval.feedback import Groundedness
grounded = Groundedness(groundedness_provider=provider)

In [15]:
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons,
             name="Groundedness"
            )
    .on(context_selection)
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


## Evaluation

In [16]:
from trulens_eval import TruLlama
from trulens_eval import FeedbackMode

tru_recorder = TruLlama(
    sentence_window_engine,
    app_id="App_1",
    feedbacks=[
        f_qa_relevance,
        f_qs_relevance,
        f_groundedness
    ]
)

In [17]:
eval_questions = []
with open('eval_questions_careerinAI.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)
eval_questions

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?']

In [18]:
for question in eval_questions[0:1]:
    with tru_recorder as recording:
        sentence_window_engine.query(question)

In [19]:
tru_recorder

TruLlama(tru_class_info=trulens_eval.tru_llama.TruLlama, app_id='App_1', tags='-', metadata={}, feedback_definitions=[], feedback_mode=<FeedbackMode.WITH_APP_THREAD: 'with_app_thread'>, root_class=llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine, app=<llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine object at 0x0000022A0048EE90>, initial_app_loader_dump=None, app_extra_json={}, feedbacks=[Feedback(tru_class_info=trulens_eval.feedback.feedback.Feedback, implementation=Method(obj=Obj(cls=trulens_eval.feedback.provider.openai.OpenAI, id=2379462008064, init_bindings=Bindings(args=(), kwargs={'tru_class_info': {'name': 'OpenAI', 'module': {'package_name': 'trulens_eval.feedback.provider', 'module_name': 'trulens_eval.feedback.provider.openai'}, 'bases': [{'name': 'OpenAI', 'module': {'package_name': 'trulens_eval.feedback.provider', 'module_name': 'trulens_eval.feedback.provider.openai'}, 'bases': None}, {'name': 'LLMProvider', 'module': {'package_nam

In [20]:
records, feedback = tru.get_records_and_feedback(app_ids=[])
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,latency,total_tokens,total_cost
0,App_1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_eea60e41b3f2c51f2da4bf85f37aeaaf,"""What are the keys to building a career in AI?""","""To build a career in AI, it is important to h...",-,"{""record_id"": ""record_hash_eea60e41b3f2c51f2da...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-30T10:12:10.171106"", ""...",2023-12-30T10:12:17.334884,7,441,0.000713


In [21]:
import pandas as pd

pd.set_option("display.max_colwidth", None)
records[["input", "output"] + feedback]

,input,output
0,"""What are the keys to building a career in AI?""","""To build a career in AI, it is important to have a strong understanding of the field and its various applications. Developing expertise in programming languages such as Python and R, as well as machine learning algorithms and techniques, is crucial. Additionally, staying updated with the latest advancements in AI and continuously learning and improving one's skills is essential. Building a strong network within the AI community and seeking out opportunities for practical experience, such as internships or research projects, can also greatly contribute to a successful career in AI."""


In [22]:
records, feedback = tru.get_records_and_feedback(app_ids=[])
records

app_id  \
0  App_1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      app_json  \
0  {"tru_class_info": {"name": "TruLlama", "module": {"package_name": "trulens_eval", "module_name": "trulens_eval.tru_llama"}, "bases": [{"name": "TruLlama", "module": {"package_name": "trulens_eval", "module_name": "trulens_eval.tru_llama"}, "bases": null}, {"name": "App", "module": {"package_name": "trulens_eval", "module_name": "trulens_eval.app"}, "bases": null}, {"name": "AppDefinition", "module": {"package_name": "trulens_eval", "module_name": "trulens_eval.schema"}, "bases": null}, {"name": "SerialModel", "module": {"package_name": "trulens_eval.utils", "module_name": "trulens_eval.utils.serial"}, "bases": null}, {"name": "WithClassInfo", "module": {"package_name": "trulens_eval.utils", "module_name": "trulens_eval.utils.pyschema"}, "bases": null}, {"name": "BaseModel", "module": {"package_name": "pydantic", "module_name": "pydantic.main"}, "bases": null}, {"name": "WithInstrumentCallbacks", "module": {"package_name": "trulens_eval", "module_name": "trulens_eval.instruments"}, "bases": null}, {"name": "Hashable", "module": {"package_name": "collections", "module_name": "collections.abc"}, "bases": null}, {"name": "Generic", "module": {"package_name": "", "module_name": "typing"}, "bases": null}, {"name": "object", "module": {"package_name": "", "module_name": "builtins"}, "bases": null}]}, "app_id": "App_1", "tags": "-", "metadata": {}, "feedback_definitions": [], "feedback_mode": "with_app_thread", "root_class": {"name": "RetrieverQueryEngine", "module": {"package_name": "llama_index.query_engine", "module_name": "llama_index.query_engine.retriever_query_engine"}, "bases": null}, "app": {"__tru_non_serialized_object": {"cls": {"name": "RetrieverQueryEngine", "module": {"package_name": "llama_index.query_engine", "module_name": "llama_index.query_engine.retriever_query_engine"}, "bases": null}, "id": 2379416661648, "init_bindings": null}}, "initial_app_loader_dump": null, "app_extra_json": {}}   

                                                                    type  \
0  RetrieverQueryEngine(

In [23]:
tru.get_leaderboard(app_ids=[])

,latency,total_cost
app_id,,
App_1,7.0,0.000713


In [24]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://localhost:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [25]:
tru.dashboard_urls()

TypeError: 'str' object is not callable